In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D,MaxPooling2D,Flatten
import pandas as pd


In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e02d0a6d0dc141fabfd2002c0d134d4d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cWFD9YuRklV5IGgnaKScG6-zkPhPhuK6oJbBTB2sVrFJ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_e02d0a6d0dc141fabfd2002c0d134d4d.get_object(Bucket='digitalnaturalist-donotdelete-pr-4q57xffjqcpv6h', Key='New.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
ls

New/


In [5]:
pwd

'/home/wsuser/work'

In [6]:
import os
filenames=os.listdir('/home/wsuser/work/New')

In [7]:
from tensorflow.keras.preprocessing .image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
x_train=train_datagen.flow_from_directory(r"/home/wsuser/work/New/traindata",target_size = (64,64),batch_size = 32,class_mode = "categorical")
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/New/testdata",target_size = (64,64),batch_size = 32,class_mode = "categorical")

Found 8305 images belonging to 9 classes.
Found 900 images belonging to 9 classes.


In [9]:
x_train.class_indices

{'ANTBIRD': 0,
 'PEACOCK': 1,
 'WILD TURKEY': 2,
 'gatto': 3,
 'mucca': 4,
 'pecora': 5,
 'rose': 6,
 'sunflower': 7,
 'tulip': 8}

In [10]:
model=Sequential()

In [11]:
model.add(Convolution2D(32,(3,3),input_shape = (64,64,3)))

In [12]:
model.add(MaxPooling2D((2,2)))

In [13]:
model.add(Flatten())

In [14]:
model.add(Dense(units = 128, kernel_initializer= "random_uniform",activation = "relu"))

In [15]:
model.add(Dense(units = 9, kernel_initializer= "random_uniform",activation = "softmax"))

In [16]:
model.compile(optimizer= "rmsprop",loss = "categorical_crossentropy" , metrics =["accuracy"])

In [17]:
model.fit(x_train,steps_per_epoch =70 ,epochs = 50, validation_data = x_test,validation_steps =len(x_test) )

Epoch 1/50
70/70 [==============================] - 13s 183ms/step - loss: 1.9486 - accuracy: 0.3762 - val_loss: 2.0204 - val_accuracy: 0.3400
Epoch 2/50
70/70 [==============================] - 13s 182ms/step - loss: 1.4320 - accuracy: 0.4903 - val_loss: 1.8230 - val_accuracy: 0.3933
Epoch 3/50
70/70 [==============================] - 14s 200ms/step - loss: 1.2951 - accuracy: 0.5134 - val_loss: 1.7037 - val_accuracy: 0.4533
Epoch 4/50
70/70 [==============================] - 14s 196ms/step - loss: 1.2639 - accuracy: 0.5330 - val_loss: 1.4981 - val_accuracy: 0.4911
Epoch 5/50
70/70 [==============================] - 14s 204ms/step - loss: 1.1741 - accuracy: 0.5598 - val_loss: 1.9463 - val_accuracy: 0.3856
Epoch 6/50
70/70 [==============================] - 14s 197ms/step - loss: 1.1539 - accuracy: 0.5786 - val_loss: 1.6529 - val_accuracy: 0.4878
Epoch 7/50
70/70 [==============================] - 12s 169ms/step - loss: 1.1113 - accuracy: 0.5964 - val_loss: 1.4812 - val_accuracy: 0.5656

In [19]:
model.save("natur1.h5")

In [20]:
!tar -zcvf digital_naturalist_model.tgz natur1.h5

natur1.h5


In [21]:
ls -1

digital_naturalist_model.tgz
natur1.h5
New/


In [22]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 21.7 MB/s eta 0:00:01


In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"1Xx4o4NnwtB9Nr7_kRr_3vtLm62vYVTaYl51ejQZCTaL"
}
client = APIClient(wml_credentials)

In [24]:
client = APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [26]:
space_uid=guid_from_space_name(client,'digitalneutralist')
print("Space UID="+space_uid)

Space UID=b14255e2-4514-4f50-aba3-a8a717ae2c36


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [30]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [31]:
model_details = client.repository.store_model(model='digital_naturalist_model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

In [32]:
model_id

'2a138552-a46b-4180-9e4b-f285d9bc6cfc'

In [33]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [36]:
from keras.models import load_model
from keras.preprocessing import image